The following project is for a learning purpose and the data/resource is from University of Massachusetts and EliteDataScience.

Wine Taster project is to develop a ML program that can distinguish from one wine to another. The project is to introduce ourselves into the sklearn library, which contains many useful statistical tools and algorithms for machine learning, or to be specific in this case, prediction.

Since this project is a learning experience for ML with sklearn, we are weeding out a lot of exploraratory data analysis. I am planning on doing another project soon with sklearn and that will include extensive explorartory data analysis. Follow that project for exploraroty data analysis. 

Importing Libraries and Modules

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib

Loading Data Sets and Data Wrangling

In [2]:
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(dataset_url)

In [5]:
print(data.head())

  fixed acidity;"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"
0   7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5                                                                                                                     
1   7.8;0.88;0;2.6;0.098;25;67;0.9968;3.2;0.68;9.8;5                                                                                                                     
2  7.8;0.76;0.04;2.3;0.092;15;54;0.997;3.26;0.65;...                                                                                                                     
3  11.2;0.28;0.56;1.9;0.075;17;60;0.998;3.16;0.58...                                                                                                                     
4   7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5                                                                                                  

In [6]:
data = pd.read_csv(dataset_url, sep=';')
print(data.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [8]:
print(data.shape)

(1599, 12)


In [9]:
print(data.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000         

Splitting Data into training sets and test sets

In [10]:
# Separate target from training features
y = data.quality
X = data.drop('quality', axis=1)

In [11]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)

We'll set aside 20% of the data as a test set for evaluating our model. We also set an arbitrary "random state" (a.k.a. seed) so that we can reproduce our results.

Finally, it's good practice to stratify the sample by the target variable. This will ensure the training set looks similar to the test set, making the evaluation metrics more reliable.

Declare data preprocessing steps using Transformer API
The Transformer API allows you to "fit" a preprocessing step using the training data the same way you'd fit a model and then use the same transformation on future data sets.

Here's what that process looks like:

Fit the transformer on the training set (saving the means and standard deviations)
Apply the transformer to the training set (scaling the training data)
Apply the transformer to the test set (using the same means and standard deviations)

This makes the final estimate of model performance more realistic, and it allows to insert the preprocessing steps into a cross-validation pipeline

In [12]:
# Fitting the Transformer APT
scaler = preprocessing.StandardScaler().fit(X_train)
# The scaler object has the saved means and standard deviations for each feature in the training set.

In [16]:
# Applying transformer to training data
X_train_scaled = scaler.transform(X_train)
 
print(X_train_scaled.mean(axis=0))
 
print(X_train_scaled.std(axis=0))

[  1.16664562e-16  -3.05550043e-17  -8.47206937e-17  -2.22218213e-17
   2.22218213e-17  -6.38877362e-17  -4.16659149e-18  -2.54439854e-15
  -8.70817622e-16  -4.08325966e-16  -1.17220107e-15]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [17]:
# Applying transformer to test data
X_test_scaled = scaler.transform(X_test)
 
print(X_test_scaled.mean(axis=0))
 
print(X_test_scaled.std(axis=0))

[ 0.02776704  0.02592492 -0.03078587 -0.03137977 -0.00471876 -0.04413827
 -0.02414174 -0.00293273 -0.00467444 -0.10894663  0.01043391]
[ 1.02160495  1.00135689  0.97456598  0.91099054  0.86716698  0.94193125
  1.03673213  1.03145119  0.95734849  0.83829505  1.0286218 ]


Notice how the scaled features in the test set are not perfectly centered at zero with unit variance! This is exactly what we'd expect, as we're transforming the test set using the means from the training set, not from the test set itself.

In practice, when we set up the cross-validation pipeline, we won't even need to manually fit the Transformer API. Instead, we'll simply declare the class object, like so:

In [18]:
# Pipeline with preprocessing and model
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))
# A modeling pipeline that first transforms the data using StandardScaler() and then fits a model using a random forest regressor.

Declare hyperparameters to tune

Hypterparamters?
There are two types of parameters we need to worry about: model parameters and hyperparameters. Models parameters can be learned directly from the data (i.e. regression coefficients), while hyperparameters cannot.
Hyperparameters express "higher-level" structural information about the model, and they are typically set before training the model.

In [19]:
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

Tune model using a cross-validation pipeline

In [21]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
# Fit and tune model
clf.fit(X_train, y_train)

# Now, check to see the best set of parameters found using CV:
print(clf.best_params_)

{'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'sqrt'}


Refit on the entire training set

In [24]:
# Confirm model will be retained
print(clf.refit)

True


Evaluate model pipeline on test data

In [25]:
# Predict a new set of data
y_pred = clf.predict(X_test)

In [26]:
# Check if the performance is good enough

print(r2_score(y_test, y_pred))
 
print(mean_squared_error(y_test, y_pred))

0.469763545009
0.3421475


Are the results satisfying ? We have only conducted cross-validation here and we have so many more algorithms and models to check this data on. So, I would not say that this is a complete but requires a bit more data analysis in order to be sure that this is enough.

Save model for future use

In [27]:
joblib.dump(clf, 'rf_regressor.pkl')
# To load: clf2 = joblib.load('rf_regressor.pkl')

['rf_regressor.pkl']